In [1]:
from IPython.display import clear_output
clear_output()

In [2]:
!pip install tqdm 
!pip install wandb -qqq
!pip install timm
!pip install --upgrade --force-reinstall --no-deps kaggle
!pip install einops
clear_output()

In [3]:
!pip install -U albumentations

In [4]:
!git clone https://github.com/Bjarten/early-stopping-pytorch

In [5]:
import sys
sys.path.append('./early-stopping-pytorch')

In [6]:
import os
import gc
import pandas as pd
import numpy as np
import wandb
import cv2
import timm
from timm.models import load_checkpoint
from tqdm.notebook import trange, tqdm
import torchvision.models as models
from einops import rearrange, reduce, repeat
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch.optim import AdamW, Adam
import albumentations as A
from albumentations.pytorch import ToTensorV2
from pytorchtools import EarlyStopping
from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import ShuffleSplit
from datetime import datetime

In [7]:
Config = {
    'CSV_PATH': "../input/petfinder-pawpularity-score/train.csv",
    'IMG_PATH': "../input/petfinder-pawpularity-score/train",
    'TEST_PATH': '../input/petfinder-pawpularity-score/test',
    'N_SPLITS': 5,
    'TRAIN_BS': 64,
    'VALID_BS': 64,
    'N_EPOCHS': 10,
    'NUM_WORKERS': 4,
    'LR': 1e-5,
    'ARCH': 'ResNet18',
    'OPTIM': "AdamW",
    'SCH_STEP': 4,
    'GAMMA': 0.1,
    'SCHEDULER': 'CosineWarm',
    "T_0": 20,
    "η_min": 1e-6,
    'PATIENCE': 3,
    'LOSS': "RMSE",
    'IMG_SIZE': 224,
    'DEVICE': "cuda",
    'infra': "Colab",
    'competition': 'pawpularity',
    '_wandb_kernel': 'tanaym',
    "wandb": True,
}

In [8]:
class PawsDataset(Dataset):
    def __init__(self, df, config=Config, augments=None, target_transform=None, is_test=False,is_val=False):
        if type(df) == pd.core.frame.DataFrame:
            self.df = df
        else:
            self.df = pd.read_csv(df)
        self.config = config
        self.transform = augments
        self.target_transform = target_transform
        self.is_test = is_test
        self.is_val = is_val
        
        self.img_paths = self._get_img_paths(self.df, self.config)
        self.meta_feats = self._get_meta_feats(self.df, self.is_test)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.img_paths[idx]
        #image = read_image(img_name)
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image = rearrange(image, 'c x y -> x y c')
        label = self.df.iloc[idx, -1]
        label = np.array([label])
        if self.transform:
            image = self.transform(image=image)['image']
        sample = {'image': image, 'label': label}
        return sample
 
    def __len__(self):
        return len(self.df)

    def _get_img_paths(self, df, config):
        """
        Returns the image paths in a list
        """
        if self.is_val:
            imgs = df['Id'].apply(lambda x: os.path.join(config['TEST_PATH'], x + ".jpg")).tolist()
        else:
            imgs = df['Id'].apply(lambda x: os.path.join(config['IMG_PATH'], x + ".jpg")).tolist()
        return imgs
    
    def _get_meta_feats(self, df, is_test):
        """
        Returns the meta features in a df
        """
        if self.is_test or self.is_val:
            meta = self.df.drop(['Id'], axis=1)
            return meta
        else:
            meta = self.df.drop(['Id', 'Pawpularity'], axis=1)
            return meta

In [9]:
class GetDate():
    def __init__(self):
        self.currentMinute = datetime.now().minute
        self.currentHour = datetime.now().hour
        self.currentDay = datetime.now().day
        self.currentMonth = datetime.now().month
        self.currentYear = datetime.now().year

In [10]:
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps
        
    def forward(self,yhat,y):
        loss = torch.sqrt(self.mse(yhat,y) + self.eps)
        return loss

In [11]:
def plot_loss_val(training_losses,valid_losses):
    plt.figure(figsize=(10,5))
    plt.title("Training and Validation Loss")
    plt.plot(training_losses,label="train")
    plt.plot(valid_losses,label="val")
    plt.xlabel("iterations")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

In [12]:
paws_dataset = PawsDataset(df=Config['CSV_PATH'],)
paws_dataset.df

In [13]:
def plot_image(image):
    """Show image with landmarks"""
    plt.imshow(image)

    plt.pause(0.001)  # pause a bit so that plots are updated


In [14]:
fig = plt.figure()

for i in tqdm(range(5)):
    sample = paws_dataset[i]
    image = sample['image'] 
    print(f"{i} {image.shape} Pawpularity:{sample['label']}")

    if image.shape[0] == 3:
        image = rearrange(image,'c x y -> x y c')
    ax = plt.subplot(1, 5, i + 1)
    plt.tight_layout()
    ax.set_title('Sample #{}'.format(i))
    ax.axis('off')
    
    plot_image(image)


In [15]:
train_aug = A.Compose(
    [
        A.SmallestMaxSize(max_size=256),
        A.Flip(p=0.5),
        A.GaussNoise(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.RandomCrop(height=224, width=224),
        A.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, p=0.1),
        A.RandomBrightnessContrast(p=0.4),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)

test_aug = A.Compose(
    [
        A.SmallestMaxSize(max_size=256),
        A.CenterCrop(height=224, width=224),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.pytorch.transforms.ToTensorV2(),
    ]
)

In [16]:
X = paws_dataset.df.iloc[:,:-1]
y = paws_dataset.df['Pawpularity']
rs = ShuffleSplit(n_splits=1, test_size=.2, random_state=42)
rs.get_n_splits(X)

print(rs)

for train_index, test_index in rs.split(X,y):
    print("TRAIN:", train_index, "TEST:", test_index)

In [17]:
training_data = PawsDataset(df=paws_dataset.df.iloc[train_index],config=Config,augments=train_aug)

test_data = PawsDataset(df=paws_dataset.df.iloc[test_index],config=Config,augments=test_aug,is_test=True)

In [18]:
trainloader = DataLoader(training_data, batch_size=Config['TRAIN_BS'], shuffle=True)

testloader = DataLoader(test_data,batch_size=Config['TRAIN_BS'],shuffle=True)

In [19]:
model_name = Config['ARCH']

model = models.resnet18(pretrained=True)
model

In [20]:
model.fc = nn.Linear(model.fc.in_features,1)

In [21]:
model

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)
model.to(device)

clear_output()

In [23]:
def train(epoch):
    running_loss = 0.0
    loss_values = []
    model.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for i, data in enumerate(tepoch):
            tepoch.set_description(f"Epoch {epoch}")
            inputs, labels = data['image'], data['label']
            labels = labels.to(torch.float32)
            inputs = inputs.to(device) 
            labels = labels.to(device)
            # zero the parameter gradients
            scheduler2.step(epoch+i/len(tepoch))
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion2(outputs, labels)
            #rmseloss = criterion2(outputs,labels)
            loss.backward()
            optimizer.step()
            # print statistics 
    
            running_loss += loss.item()
            loss_values.append(loss.item())
            if i % 64 == 0 and i>0:    # every 64 mini-batches
                train_loss = running_loss/i
                print(i)
                print(f'Training loss: {train_loss}')
        train_loss = sum(loss_values) /len(tepoch)
        
        gc.collect()
        torch.cuda.empty_cache()
        return train_loss

In [24]:
def test(epoch):
    losslis=[]
    test_loss = 0.0
    # since we're not training, we don't need to calculate the gradients for our outputs
    model.eval()
    with torch.no_grad():
        for data in tqdm(testloader):
            inputs, labels = data['image'], data['label']
            # calculate outputs by running images through the network
            labels = labels.to(torch.float32)
            inputs = inputs.to(device) 
            labels = labels.to(device)
            outputs = model(inputs)
            # print(outputs)
            # print(labels)
            loss = criterion2(outputs,labels)

            test_loss += loss.item()
            losslis.append(loss.item())
            #writer.add_scalar('Test/Loss', test_loss, epoch)  
        test_loss = test_loss/len(testloader)
        print(f'Validation loss: {test_loss}')
    gc.collect()
    torch.cuda.empty_cache()
    return test_loss

In [25]:
criterion1 = nn.BCEWithLogitsLoss()
criterion2 = RMSELoss()
optimizer = AdamW(model.parameters(), lr=Config['LR'] )

In [26]:
scheduler1 = torch.optim.lr_scheduler.StepLR(optimizer, step_size=Config['SCH_STEP'],
                gamma=Config['GAMMA']) 
scheduler2 = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=Config['T_0'],
                eta_min=Config['η_min'])

In [27]:
scheduler_name = f"{Config['SCHEDULER']}_{Config['SCH_STEP']}_{Config['GAMMA']}"
early_stopping = EarlyStopping(patience=Config['PATIENCE'], verbose=True)

In [28]:
from kaggle_secrets import UserSecretsClient
secret_label = "wandb"
secret_value = UserSecretsClient().get_secret(secret_label)

In [29]:
if Config['wandb']:
  wandb.init(project="pawpularity_kaggle_final",entity="edgarlcs",config=Config,name=f'{model_name}_{scheduler_name}_epoch{Config["N_EPOCHS"]}_{Config["TRAIN_BS"]}')
  wandb.watch(model)

In [30]:
epochs = Config['N_EPOCHS']
training_losses = []
valid_losses = []

for epoch in tqdm(range(1,epochs+1)):
    train_loss = train(epoch)
    test_loss = test(epoch)
    
    #scheduler2.step()
    early_stopping(test_loss, model)
        
    if early_stopping.early_stop:
      print("Early stopping")
      break
    if Config['wandb']:
      wandb.log({
        "Epoch": epoch,
        "Train Loss": train_loss,
        "Valid Loss": test_loss})
    else:
      training_losses.append(train_loss)
      valid_losses.append(test_loss)
print('Finished Training')

In [31]:
if Config['wandb'] == False:
  plot_loss_val(training_losses,valid_losses)

In [32]:
now = GetDate()